# Azure Search Index Generator

**Executive summary**

This notebook provides simple instructions on creating and using your first search application that will work for your data.

Its essential purpose is to give a user-friendly point to start getting your first search engine working!

The only prerequisite is an MS Azure Subscription account. 

The notebook begins by explaining the prerequisites to start setting up the index. Next, it walks you through creating a search index on an Azure Search Instance. Then, it will provide four ways of accessing the results from the search index. Finally, it will show a powerful enhancement of your index, allowing it to return answers to questions formulated in natural language.

**Detailed contents:**

**1. Required assets** 
- Creating a storage account and a blob storage
    - Storage account connection string
- Uploading JSONs to the blob storage
- Creating an Azure Search Instance
    - Azure Search Key

**2. Setting up the index**
- Connecting to a data source  
- Creating an empty index
    - Index schema
    - Creating an index
- Indexing the documents

**3. Ways to search the index**
- Web interface 
- Generate your separate app! 
- Search client
- RESP API 

**4. Semantic enhancement** 
- Enable semantic search
- How to create semantic configurations
- How does semantic enhancement effect the search?

# 1. Required assets


## Creating a storage account and a blob storage

*Azure Storage* is a service that allows to create various types of data storage. 
https://azure.microsoft.com/en-us/products/category/storage/

Here you may find detailed instructions on how to create it:
https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create?tabs=azure-portal

**Where to find the storage connection string?**

On the left pane, below the Azure Storage account name, pick 'Access keys' and then copy one of the connection strings:

![Index9](./jpg/index9.jpeg)

To run this notebook, you also need to create *a blob container* from which your documents will be indexed.

This type of storage allows to keep significant amounts of unstructured data:
https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction

To do it manually, navigate to the description of "Containers" and click the button on the upper-right pane:


![Index7](./jpg/index7.jpeg)

## Uploading JSONs to the blob storage

The easiest way to add your documents to a blob storage is manual. Inside the discription of your container, click "upload":

![Index8](./jpg/index8.jpeg)


## Creating an Azure Search Instance

*Azure Search* is a service instance that allows to create and maintain one or multiple search indexes.  

Here you may find detailed instructions on how to create it:
https://docs.microsoft.com/en-us/azure/search/search-what-is-azure-search#how-to-get-started

To perform the following operations on index creation, you need to save:
- the name of your Azure search service (for example, "*my-azure-search-index*")
- admin key 

**Where to find the admin key?**

On the left pane below the Azure Search Instance name, pick 'Keys'.

Then copy one of the admin keys: 

![Index6](./jpg/index6.jpeg)

Remark: Besides the admin keys, it is possible to generate 'Query' keys that would give only limited permissions to query the data in the index. 

# 2. Setting up the index 
### 2.1 Put the connection details together 

Below you need to enter the connection parameters you got in the previous section.

From the Azure Search instance:
- the name of the service
- administration key 

From the storage account:
- connection string
- the name of the blob container, where you have your JSONs

In [1]:
azure_search_service = "my-first-search"
admin_key = ""

#blob storage data 
blob_connection_string = ""
container = "my-first-blob-storage"

*Remark*: The best practice here is to use a .env file to store all keys as environment variables. It will prevent the accidental push of secrets. For more info: https://pypi.org/project/python-dotenv/#getting-started. This notebook explicitly provides keys for the sake of simplicity for the user to get started. 

### 2.2 Name your Azure Search components

Make up the names for the *data source* (essentially, the link to the blob storage you have just created) and the *Azure Search Indexer* (the indexer instance for your search engine). 

The name of your *Azure Search Index*  (the schema for your search engine) is determined based on the index schema by default, but you can also modify it further. 

Important: Azure Search components' names must only contain lowercase letters, digits, or dashes, cannot start or end with dashes, and are limited to 128 characters.

In [2]:
AS_data_source = "my-first-datasource"
AS_azure_search_indexer = "my-first-indexer"

### Import dependencies


In [4]:
import json
import os
from requests import post, put 
import pandas as pd

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

def resp_explicit_codes(resp):
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")

    if resp.status_code == 400:
        print(resp.text)   

def request_to_azure_search(json_file, feature, azure_search_service, name="", api_version='2021-04-30-Preview'):
    """
        This function creates a rest call to Azure search indexer with multiple endpoints
        * index - to create a new Index based on configuration
        * indexers - create a new Indexer based on configuration
        * datasources - create a new DataSource based on configuration


        json_file (_type_): configuation to pass with the call
        feature (_type_): type of endpoint to execute
        name (str, optional): Name of the operation to pass. Defaults to "".
        api_version (str, optional): API version to use in calls. Defaults to '2021-04-30-Preview'.
    """
    if feature not in ['indexes', 'datasources', 'indexers']:
        print('Error: this feature is not supported.')
        return 
    
    headers = {
    "api-key": admin_key,     
    "Content-Type": "application/json",

    }
    endpoint = f"https://{azure_search_service}.search.windows.net/".format(azure_search_service)
        
    if (name==""):
        try: 
            name_feature = json_file['name']
            url = endpoint +"/" + feature + f"?api-version={api_version}"
        except Exception as e:
            print('The name of the feature is not specified')
            return
    else:
        name_feature = name
        url = endpoint +"/" + feature + "/" + f"{name}" + f"?api-version={api_version}"
        
    try:
        resp = post(url=url, json=json_file, headers=headers)
        
        resp_explicit_codes(resp)   

        if resp.status_code == 201:
            print("Success creating the component: " + name_feature + ".")

    except Exception as e:
        print('Exception for the feature' +  name_feature, e, resp.text)

*API-version* is the version of Search REST APIs. 

API-versions may differ in terms of the allowed options and REST interface. 
This notebook employs the '*2021-04-30-Preview*' version because, during our investigation preceding the release of this tool, it was established that it is the most reliable performance, especially regarding the semantic search. In particular, it is the only version allowing the semantic search option in the web interface.

Read about other available options:
https://docs.microsoft.com/en-us/rest/api/searchservice/search-service-api-versions

### 2.3 Creating the Azure Search Index components 

Now you are ready to create all necessary parts of your first search engine!

### 2.3.1 Connecting a data source

In [21]:
datasource_initialization_details = {
    "name" : AS_data_source,
    "type" : "azureblob",
    "credentials" : {"connectionString": blob_connection_string},
    "container" : { "name" :  container}
}

try:
    request_to_azure_search(datasource_initialization_details, feature="datasources", azure_search_service = azure_search_service)
except Exception as e:
    print(e)

Success creating the component: my-first-datasource.


### 2.3.2. Create an empty index 

#### Index schema

First, you must create an empty index with a structure corresponding to your data. 

The Azure Search instance's visual interface allows to specify the structure manually. 

![Index1](./jpg/index1.jpeg)

This approach requires specifying each field separately:   

![Index2](./jpg/index2.jpeg)

#### Index schema as a file  

Manual approach might be tedious in many cases. A more convenient method to specify a JSON index schema file. 

Importantly, this file must have a particular structure: 

In [ ]:
{  
  "name": (optional on PUT; required on POST) "Name of the index",  
  "fields": [  
    {  
      "name": "name_of_field",  
      "type": "Edm.String | Edm.Int32 | Edm.Int64 | Edm.Double | Edm.Boolean | Edm.DateTimeOffset | Edm.GeographyPoint | Edm.ComplexType | Collection(Edm.String) | Collection(Edm.Int32) | Collection(Edm.Int64) | Collection(Edm.Double) | Collection(Edm.Boolean) | Collection(Edm.DateTimeOffset) | Collection(Edm.GeographyPoint) | Collection(Edm.ComplexType)",  
      "searchable": true (default where applicable) | false (only Edm.String and Collection(Edm.String) fields can be searchable),  
      "filterable": true (default) | false,  
      "sortable": true (default where applicable) | false (Collection(Edm.String) fields cannot be sortable),  
      "facetable": true (default where applicable) | false (Edm.GeographyPoint fields cannot be facetable),  
      "key": true | false (default, only Edm.String fields can be keys, enable on one field only),  
      "retrievable": true (default) | false,  
      "analyzer": "name_of_analyzer_for_search_and_indexing", (only if 'searchAnalyzer' and 'indexAnalyzer' are not set)
      "searchAnalyzer": "name_of_search_analyzer", (only if 'indexAnalyzer' is set and 'analyzer' is not set)
      "indexAnalyzer": "name_of_indexing_analyzer", (only if 'searchAnalyzer' is set and 'analyzer' is not set)
      "synonymMaps": [ "name_of_synonym_map" ] (optional, only one synonym map per field is currently supported),
      "fields" : [ ... ] (optional, a list of sub-fields if this is a field of type Edm.ComplexType or Collection(Edm.ComplexType). Must be null or empty for simple fields.)
    }
  "similarity": (optional) { },
  "suggesters": (optional) [ ... ],  
  "scoringProfiles": (optional) [ ... ],  
  "analyzers":(optional) [ ... ],
  "charFilters":(optional) [ ... ],
  "tokenizers":(optional) [ ... ],
  "tokenFilters":(optional) [ ... ],
  "defaultScoringProfile": (optional) "Name of a custom scoring profile to use as the default",  
  "corsOptions": (optional) { },
  "encryptionKey":(optional) { }  
}

For more details: https://docs.microsoft.com/en-us/rest/api/searchservice/create-index

A simple sample schema file is located in *assets* directory: [*template_index_schema.json*](./assets/template_index_schema.json)

The schema corresponding to the sample JSONs is in the same directory: [*sample_index_schema.json*](./assets/sample_index_schema.json)

**In case your JSONs do not have a nested structure, you can also use [our accompanying notebook that containes an index schema file generator](./CreateIndexSchema.ipynb)**

### 2.3.3 Create the index

Here you reference your index schema file. We assume that it is in the same directory as this notebook. 

In [22]:
index_schema_file = './assets/sample_index_schema.json'

with open(index_schema_file) as f:
    index_schema = json.loads(f.read())

Based on the file the name of your index: 

In [23]:
index_schema['name']

'my-first-index'

In you want to change it, you can do it here: 

In [24]:
index_schema['name'] = 'my-first-index'

Let's create the index: 

In [26]:
try:
    request_to_azure_search(index_schema, feature="indexes", azure_search_service = azure_search_service)
except Exception as e:
    print(e)

Success creating the component: my-first-index.


For the sample JSONs, you index schema must now look the following way:

![Index3](./jpg/index3.jpeg)

### 2.3.3 Index the documents

Now there is only one step left: to add your documents to the indexer: 

In [28]:
indexer_initialization_details = {
    "name": AS_azure_search_indexer,
    "dataSourceName":  AS_data_source,
    "targetIndexName": index_schema['name'],
    "parameters": {
    "batchSize": None,
    "maxFailedItems": None,
    "maxFailedItemsPerBatch": None,
    "base64EncodeKeys": None,
    "configuration": {
      "parsingMode": "json"
    }
    }
}

try:
    request_to_azure_search(indexer_initialization_details, feature="indexers", azure_search_service = azure_search_service)
except Exception as e:
    print(e)

Success creating the component: my-first-indexer.


Congratulations! You have created your Azure Search Index. 
However, you may have to wait a couple of minutes before your documents get indexed.

After that you can perform the first search over your documents.

## 3. Accessing the index

There are multiple ways to access the index, besides the REST API. 

In [132]:
index_name = "my-first-index"

### 3.1 Web interface

The simplest one is to access the embedded search interface. For that, you need top click the name of your index under 'Indexes'. Then, you will see the search page: 

![Index10](./jpg/index10.jpeg)

### 3.2 Generate your separate app!

On the page above, you can see the button 'Create Demo App'. If you click on it, you will be proposed to download a html-page.

If you open it, you will get your search app that will query your documents:

![Index11](./jpg/index11.jpeg)

### 3.3 Search client

Finally, you can search your documents via a search client: 

In [7]:
endpoint = "https://{}.search.windows.net/".format(azure_search_service)

search_client = SearchClient(endpoint = endpoint,
                       index_name = index_name,
                       credential = AzureKeyCredential(admin_key))

Now you can access your documents using code. 

In [134]:
results =  search_client.search(search_text="the best restaurant", include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())

for r in results:
    print("{}: {} [score: {}]".format(r['HotelName'],r['Description'],r['@search.score']))
    print()

Total Documents Matching Query: 4
Triple Landscape Hotel: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services. [score: 0.7114401]

Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities. [score: 0.51262975]

Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. [score: 0.3097111]

Twin Dome Motel: The hotel is situated in a  nineteenth centur

More examples of searches can be found here: https://docs.microsoft.com/en-us/azure/search/search-get-started-python

### 3.4 REST API

The most transparent way of accessing the indexed documents is throuth REST API. 
(Here you can read more about it: https://restfulapi.net/)

In [106]:
def get_rest_query_url(azure_search_service, index_name, api_version="2021-04-30-Preview"):
    """
        This funcion returns an url in the format suitable for a REST API query request 
        azure_search_service (str): name of the Azure Search Service
        index_name (str): name of the index 
        api_version (str, optional): API version to use in calls. Defaults to '2021-04-30-Preview'.
    """
    endpoint = f"https://{azure_search_service}.search.windows.net/".format(azure_search_service)
    return endpoint +"indexes/" + f"{index_name}" + "/docs/search" + f"?api-version={api_version}"

def post_query(query_text, url):
    """
        This Funcion creates a rest call to azure search indexer to perform a query
        
        query_text (str): text of the query
        url (str): url suitable for a rest call 
        semantic_configuration (str, optional): name of the semantic configuration. Defaults to "".
        
        returns a list of dictionaries with query results
    """
    headers = {
    "api-key": admin_key,     
    "Content-Type": "application/json",
    }
    
    query = {
    "search": query_text,
    "queryType": "simple",
    }
    
    try:
        resp = post(url=url, json=query, headers=headers)

        resp_explicit_codes(resp)   
            
        if resp.status_code == 200:
            print("Query successful!\n")
            

    except Exception as e:
        print('Exception during the query: ', e)
        return
        
    return json.loads(resp.text)['value']

url = get_rest_query_url("lensearch","my-first-index")

In [130]:
results = post_query(query_text ='the best restraurant', url = url)

for r in results:
    print("{}: {} [score: {}]".format(r['HotelName'],r['Description'],r['@search.score']))
    print()

Query successful!

Triple Landscape Hotel: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services. [score: 0.7114401]

Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities. [score: 0.51262975]

Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. [score: 0.3097111]

Twin Dome Motel: The hotel is situated in a  nineteenth century plaza, which 

# 4. Semantic enhancement 

As shown in the previous section, the results of the default search setting are already pretty impressive. 

Meanwhile, there is a way to further strengthen the search results by enabling semantic enhancement. 
That option might be handy if the answer to the query is not straightforward. 

Semantic search is a powerful, cutting-edge technology. However, it is crucial to remember that it is an experimental feature. Also, one critical point to keep in mind: if your query results are expected to have many items, the semantic search reranks only the top 50 of them.

**What does it do?** 

Semantic configuration allows promoting the most semantically relevant results to the top of the list.

Notably, the following instruction will show a possible way to start. As with the rest of this notebook, its purpose is to provide you with a kick-start point. 

For a deeper dive, check this: 

https://docs.microsoft.com/en-us/azure/search/semantic-search-overview

## 4.1 Enable semantic search

First of all, you need to enable the option of semantic search:

https://docs.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search

## 4.2 Create semantic configurations

Semantic configuration can be created using the visual tools on the webpage of the index:

![Index12](./jpg/index12.jpeg)

Another way to specify a semantic configuration is to add its definition to the JSON index schema: 

In [ ]:
  "semantic": {
    "configurations": [
      {
        "name": "semconf",
        "prioritizedFields": {
          "titleField": {
            "fieldName": "HotelName"
          },
          "prioritizedContentFields": [
            {
              "fieldName": "Description"
            }
          ],
          "prioritizedKeywordsFields": [
            {
              "fieldName": "Tags"
            }
          ]
        }
      }
    ]
  }

You can find the schema that includes the definition of the semantic configuration: [*sample_index_schema_semantic.json*](./assets/sample_index_schema_semantic.json)

One search index can have multiple semantic configurations. 

While for illustrative purposes, this notebook provides the basic example of a semantic configuration with one content field (*Description*) and one keyword field (*Tags*), it is possible to create semantic configurations with a more complex structure. 

Once you have created a semantic configuration, you can allow it while creating your query in the Web interface (make sure that the API vesion you are using allows it):

![Index13](./jpg/index13.jpeg)

## 4.3 How does semantic enhancement effect the search?

(For more options for your queries, check: https://docs.microsoft.com/en-us/azure/search/semantic-how-to-query-request)

Let's modify *post_query()*, so it allows semantic enhancement:

In [133]:
def post_query(query_text, url, query_type = "simple", semantic_configuration=""):
    """
        This Funcion creates a rest call to azure search indexer to perform a query
        
        query_text (str): text of the query
        url (str): url suitable for a rest call 
        query_type (str, optional): type of the query: "simple" or "semantic". Defaults to "simple".
        semantic_configuration (str, optional): name of the semantic configuration. Defaults to "".
        
        returns a list of dictionaries with query results
    """
    headers = {
    "api-key": admin_key,     
    "Content-Type": "application/json",
    }
    
    query = {
         "search": query_text,
         "queryType": query_type,
         "queryLanguage": "en-us",
         "speller": "lexicon"    
    }
    
    if query_type == "semantic":
        query["semanticConfiguration"] = semantic_configuration 

    
    try:
        resp = post(url=url, json=query, headers=headers)
        
        resp_explicit_codes(resp)    
            
        if resp.status_code == 200:
            print("Query successful!\n")
            
        result = json.loads(resp.text)['value']
    except Exception as e:
        print('Exception during the query ', e)
        return
        
    return json.loads(resp.text)['value']
    #return resp

url = get_rest_query_url("lensearch","my-first-index")

### Example 1 

Let's start from the example we have already seen: "*the best restaurant*"

In [208]:
results = post_query(query_text ='the best restaurant', url = url)

for r in results:
    print("{}: {} [score: {}]".format(r['HotelName'], r['Description'], r['@search.score']))
    print()  

Query successful!

Triple Landscape Hotel: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services. [score: 0.7114401]

Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities. [score: 0.51262975]

Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. [score: 0.3097111]

Twin Dome Motel: The hotel is situated in a  nineteenth century plaza, which 

While the top result and the last result are straightforward, the two middle ones are probably debatable. We might expect the historical center to be a more likely place to find better restraurants, while for the case of *Sublime Cliff Hotel*.

The semantic search corrects for that: 

In [209]:
results = post_query(query_text ='the best restraurant', url = url, query_type = "semantic", semantic_configuration = "semconf")

for r in results:
    print("{}: {} [re-ranked score: {}] [score: {}]".format(r['HotelName'], r['Description'], r['@search.rerankerScore'], r['@search.score']))
    print()

Query successful!

Triple Landscape Hotel: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services. [re-ranked score: 0.9138548012000001] [score: 0.7114401]

Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. [re-ranked score: 0.8948213380000001] [score: 0.3097111]

Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities. [re-ranked score: 0.68

Also, notably, the re-ranked score accounts for that, most likely, *Triple Landscape Hotel* and *Sublime Cliff Hotel* are not so far apart regarding the likelihood of finding the best restaurant. 

## Example 2

In the previous example, the top answer to the query about the top restaurant was clear. Let's see what happens if the question may require analysis beyond the exact content of the documents in the index. Importantly, here we don't provide the precise address in the case of the semantic search. 

In [206]:
results = post_query(query_text ='most beatiful area', url = url)

for r in results:
    print("{}: {} [score: {}]".format(r['HotelName'], r['Description'], r['@search.score']))
    print()  

Query successful!

Twin Dome Motel: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. [score: 0.6160039]

Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. [score: 0.6160039]

Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan citie

This ranking seems doubtful: the first and second options have similar ranking, while we can say not much about the area given that the builing of Twin Motel has been recently reviewed.  

In [210]:
results = post_query(query_text ='most beatiful area', url = url, query_type = "semantic", semantic_configuration = "semconf")

for r in results:
    print("{}: {} [re-ranked score: {}] [score: {}]".format(r['HotelName'], r['Description'], r['@search.rerankerScore'], r['@search.score']))
    print()

Query successful!

Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. [re-ranked score: 0.36888724780000004] [score: 0.6160039]

Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities. [re-ranked score: 0.334203658] [score: 0.26286605]

Twin Dome Motel: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which 

Now, the ranking feels more meaningful: the clear winner *Sublime Cliff Hotel* is on top, and *Twin Dome Motel* is the last. 

Notably, *Triple Landscape Hotel* has been excluded in both cases. 